### Подключим spark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

spark = SparkSession.builder \
    .appName("Spark SQL with PostgreSQL") \
    .config("spark.jars", "postgresql-42.6.0.jar, clickhouse-jdbc-0.4.6.jar") \
    .getOrCreate()

pg_jdbc_url = "jdbc:postgresql://postgres:5432/spark_db"
pg_properties = {
    "user": "spark_user",
    "password": "spark_password",
    "driver": "org.postgresql.Driver"
}

ch_jdbc_url = "jdbc:clickhouse://clickhouse:8123/default"
ch_properties = {
    "driver": "com.clickhouse.jdbc.ClickHouseDriver",
    "user": "custom_user",
    "password": "custom_password"
}

### Преобразование данных из mock_data в таблицы фактов и измерений

In [2]:
from uuid import uuid4
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
mock_data = spark.read.jdbc(url=pg_jdbc_url, table="mock_data", properties=pg_properties)
mock_data.toPandas().head()

,id,customer_first_name,customer_last_name,customer_age,customer_email,customer_country,customer_postal_code,customer_pet_type,customer_pet_name,customer_pet_breed,...,product_reviews,product_release_date,product_expiry_date,supplier_name,supplier_contact,supplier_email,supplier_phone,supplier_address,supplier_city,supplier_country
0,1,Barron,Rawlyns,61,bmassingham0@army.mil,China,None,cat,Priscella,Labrador Retriever,...,97,2011-10-19,2028-10-21,Tagcat,Bevan Massingham,bmassingham0@unblog.fr,914-877-7062,Suite 25,Kletek,China
1,2,Ham,Knowller,78,cscudder1@time.com,Poland,73-115,bird,Dalenna,Labrador Retriever,...,37,2019-04-17,2028-02-29,Livetube,Candide Scudder,cscudder1@sbwire.com,863-319-5653,18th Floor,Santa Cruz Balanyá,Poland
2,3,Farleigh,Langley,71,vhuxter2@fotki.com,Poland,56-160,bird,Aldridge,Parakeet,...,218,2010-02-03,2023-09-03,Photobug,Vinson Huxter,vhuxter2@slate.com,434-817-1275,Apt 96,Huangpu,Samoa
3,4,Shae,Debling,28,bbullier3@bravesites.com,Sri Lanka,81700,cat,Beverie,Labrador Retriever,...,959,2019-08-22,2026-03-29,Janyx,Bald Bullier,bbullier3@dot.gov,515-756-7392,12th Floor,Chengzhong,Indonesia
4,5,Lay,Twatt,25,sedgar4@smugmug.com,Botswana,None,bird,Sydelle,Siamese,...,130,2018-12-09,2030-05-10,Lazz,Sydelle Edgar,sedgar4@reverbnation.com,861-980-8811,Suite 79,Lanxi,China


In [4]:
dim_customer = mock_data.select(
    col("customer_first_name").alias("first_name"),
    col("customer_last_name").alias("last_name"),
    col("customer_age").alias("age"),
    col("customer_email").alias("email"),
    col("customer_country").alias("country"),
    col("customer_postal_code").alias("postal_code")
).distinct()

dim_customer = dim_customer.withColumn("id", expr("uuid()"))
dim_customer = dim_customer.selectExpr(
    "id", 
    "first_name", 
    "last_name", 
    "age", 
    "email", 
    "country", 
    "postal_code"
)


print(dim_customer.toPandas().shape)
dim_customer.toPandas().head()

(10000, 7)


,id,first_name,last_name,age,email,country,postal_code
0,96d20c19-1a54-4352-85b6-ee68fb86460d,Ileana,Bowness,18,wlemmanbieiw@ox.ac.uk,Canada,B0K
1,181e0642-a46a-4766-b227-e93365980c19,Rossie,Botte,38,lshahji@toplist.cz,Russia,399445
2,da342473-221b-41f9-8601-b36568ab8f4f,Arnuad,Frift,29,eschankelborgqa@google.de,Philippines,1706
3,97c9e7d1-9ef5-4ad1-b25f-a16dc2c4db5e,Ruben,Godmer,46,cburles15@typepad.com,Macedonia,1250
4,9996af53-2acd-441e-910d-fec038559629,Iolanthe,Duffill,37,mharhoff1t@berkeley.edu,France,69303 CEDEX 07


In [5]:
dim_seller = mock_data.select(
    col("seller_first_name").alias("first_name"),
    col("seller_last_name").alias("last_name"),
    col("seller_email").alias("email"),
    col("seller_country").alias("country"),
    col("seller_postal_code").alias("postal_code")
).distinct()

dim_seller = dim_seller.withColumn("id", expr("uuid()"))
dim_seller = dim_seller.selectExpr(
    "id", 
    "first_name", 
    "last_name",  
    "email", 
    "country", 
    "postal_code"
)


print(dim_seller.toPandas().shape)
dim_seller.toPandas().head()

(10000, 6)


,id,first_name,last_name,email,country,postal_code
0,0bcb92dd-f7bc-4c52-9f3f-24e573f1bcb3,Ezekiel,Rumming,erummingaj@weather.com,Russia,456143
1,6e24d67a-2cc3-44c6-b520-aad19bc794a2,Allyce,Nadin,anadine2@i2i.jp,Philippines,2822
2,53cdd6d4-215d-4a1f-bd0b-9f47c3323632,Mayer,Hinder,mhinderpo@dagondesign.com,Russia,431355
3,ec9868e8-3f46-4e7a-a259-b41dc8c05056,Urbain,Shimwall,ushimwallil@independent.co.uk,United States,87592
4,e0ed1b85-65a6-48b0-8831-143e2c4db393,Barbey,St. John,bst42@freewebs.com,Mexico,93380


In [6]:
dim_supplier = mock_data.select(
    col("supplier_name").alias("name"),
    col("supplier_contact").alias("contact"),
    col("supplier_email").alias("email"),
    col("supplier_phone").alias("phone"),
    col("supplier_address").alias("address"),
    col("supplier_city").alias("city"),
    col("supplier_country").alias("country")
).distinct()

dim_supplier = dim_supplier.withColumn("id", expr("uuid()"))
dim_supplier = dim_supplier.selectExpr(
    "id",
    "name",
    "contact",
    "email",
    "phone",
    "address",
    "city",
    "country"
)


print(dim_supplier.toPandas().shape)
dim_supplier.toPandas().head()

(10000, 8)


,id,name,contact,email,phone,address,city,country
0,d9c0f2a5-59cd-49de-928d-bd7448e20203,Oyoyo,Cirilo Hellewell,chellewellh@google.ca,484-944-2476,7th Floor,Manzë,Portugal
1,d975d5f0-c8d3-4e83-937d-8e5d3858b205,Snaptags,Evelin Alvarado,ealvaradof5@dedecms.com,120-766-6526,Apt 848,Ai Tu,Colombia
2,c613eced-0c31-4e20-9a4e-a412ab4e94dd,Wikivu,Fayette Glascott,fglascottft@infoseek.co.jp,205-362-1952,Apt 1047,Mesyagutovo,Ukraine
3,8258c1b0-105b-4bf1-a444-d265e04d5c08,Edgeify,Whitney Pomphrey,wpomphreyg7@engadget.com,485-355-8348,Suite 41,Brumadinho,Ukraine
4,ad5391d8-466a-4d9c-9047-91d7ec4a7f6b,Buzzdog,Rhianna Kneebone,rkneeboneki@merriam-webster.com,451-222-9196,Suite 88,Spassk,Armenia


In [7]:
dim_store = mock_data.select(
    col("store_name").alias("name"),
    col("store_location").alias("location"),
    col("store_city").alias("city"),
    col("store_state").alias("state"),
    col("store_country").alias("country"),
    col("store_phone").alias("phone"),
    col("store_email").alias("email")
).distinct()

dim_store = dim_store.withColumn("id", expr("uuid()"))
dim_store = dim_store.selectExpr(
    "id",
    "name",
    "location",
    "city",
    "state",
    "country",
    "phone",
    "email"
)

print(dim_store.toPandas().shape)
dim_store.toPandas().head()

(10000, 8)


,id,name,location,city,state,country,phone,email
0,80d07395-8fad-451e-b5f2-bcf3463cda08,Dabjam,Room 1734,Cayenne,GF,Greece,954-419-0001,csalmonici@fc2.com
1,a41d1cc3-ae3e-45fa-9e16-fbeaf1245a61,Lazz,PO Box 25788,Las Palmas,OAX,Sweden,696-560-4319,smacavddy6g@friendfeed.com
2,b58ae5f1-5423-4876-b228-7a4d30c7ba51,Talane,PO Box 68620,Chartres,A3,Honduras,472-917-6932,lgiacobbo5r@scientificamerican.com
3,6d45a87e-e25f-4821-8f0a-4d379bbd5c35,Bubblebox,Suite 58,Saint-Quentin,B6,Kazakhstan,863-901-5165,vturk6v@com.com
4,3ba18303-961b-421d-a2e8-3cbcb842b9f9,Voomm,Room 1145,Joliette,QC,Canada,604-183-5169,krivalants@cisco.com


In [8]:
product_data = mock_data.select(
    col("product_name").alias("name"),
    col("product_category").alias("category"),
    col("product_weight").alias("weight"),
    col("product_color").alias("color"),
    col("product_size").alias("size"),
    col("product_brand").alias("brand"),
    col("product_material").alias("material"),
    col("product_description").alias("description"),
    col("product_rating").alias("rating"),
    col("product_reviews").alias("reviews"),
    col("product_release_date").alias("release_date"),
    col("product_expiry_date").alias("expiry_date"),
    col("supplier_name"),
    col("supplier_email"),
    col("product_price").alias("price"),
    col("product_quantity").alias("quantity")
)

dim_product = product_data.join(
    dim_supplier.alias("supplier"),
    (product_data["supplier_name"] == col("supplier.name")) & 
    (product_data["supplier_email"] == col("supplier.email")),
    "inner"
).select(
    col("supplier.id").alias("supplier_id"),
    product_data["name"],
    product_data["category"],
    product_data["weight"],
    product_data["color"],
    product_data["size"],
    product_data["brand"],
    product_data["material"],
    product_data["description"],
    product_data["rating"],
    product_data["reviews"],
    product_data["release_date"],
    product_data["expiry_date"],
    product_data["price"],
    product_data["quantity"]
).distinct()

dim_product = dim_product.withColumn("id", expr("uuid()"))
dim_product = dim_product.selectExpr(
    "id",
    "supplier_id",
    "name",
    "category",
    "weight",
    "color",
    "size",
    "brand",
    "material",
    "description",
    "rating",
    "reviews",
    "release_date",
    "expiry_date",
    "price",
    "quantity"
)

print(dim_product.toPandas().shape)
dim_product.toPandas().head()

(10000, 16)


,id,supplier_id,name,category,weight,color,size,brand,material,description,rating,reviews,release_date,expiry_date,price,quantity
0,61f64b51-4c62-485b-ba3e-04198d8a4797,f6a05379-898b-4b30-b16c-204e6e3643ab,Cat Toy,Toy,11.60,Teal,Medium,Edgeclub,Aluminum,Etiam vel augue. Vestibulum rutrum rutrum nequ...,4.9,685,2020-02-08,2025-06-15,47.09,59
1,61fe8606-5a37-48cf-b0a7-bf815edf3aae,b122693a-8fcc-42cb-bfbd-d135fa4a8663,Cat Toy,Food,16.90,Indigo,Medium,Devbug,Vinyl,Nulla ut erat id mauris vulputate elementum. N...,1.3,902,2017-08-26,2027-05-10,13.72,27
2,7a214b02-dc85-4e73-aaf1-141a2b567ec6,6763ac15-dab3-4dba-8db1-d69e86728d0a,Dog Food,Toy,36.70,Fuscia,Small,Tazz,Plastic,Pellentesque at nulla. Suspendisse potenti. Cr...,1.6,343,2014-11-17,2028-03-08,61.61,40
3,a2f823e2-d7ef-46ab-bec8-be1449e5dfe8,7c10817a-969e-4ba7-abb4-08879f99ebc2,Dog Food,Food,47.00,Turquoise,Small,Reallinks,Aluminum,"Mauris enim leo, rhoncus sed, vestibulum sit a...",3.1,383,2013-04-14,2026-01-23,79.20,38
4,673939e2-125b-4f6a-afa0-f46de052551c,c495716b-4ad7-41a1-ad8f-9ccd7c141ce0,Bird Cage,Toy,47.00,Red,Medium,Ailane,Stone,Etiam vel augue. Vestibulum rutrum rutrum nequ...,3.8,889,2016-11-28,2029-10-04,80.75,89


In [9]:
dim_pet = mock_data.select(
    col("customer_email"),
    col("customer_pet_type").alias("pet_type"),
    col("customer_pet_name").alias("pet_name"),
    col("customer_pet_breed").alias("pet_breed"),
    col("pet_category").alias("pet_category")
).join(
    dim_customer.alias("customer"),
    mock_data["customer_email"] == col("customer.email"),
    "inner"
).select(
    col("customer.id").alias("customer_id"),
    col("pet_type"),
    col("pet_name"),
    col("pet_breed"),
    col("pet_category")
).distinct()

dim_pet = dim_pet.withColumn("id", expr("uuid()"))
dim_pet = dim_pet.selectExpr(
    "id",
    "customer_id",
    "pet_type",
    "pet_name",
    "pet_breed",
    "pet_category"
)

print(dim_pet.toPandas().shape)
dim_pet.toPandas().head()

(10000, 6)


,id,customer_id,pet_type,pet_name,pet_breed,pet_category
0,7c43ac30-27d6-4934-95a0-566e14a8cada,53ecd381-71e1-4e96-8bf3-8f936bbf6df8,bird,Querida,Parakeet,Reptiles
1,e18ab96a-5e74-47cc-837c-94a84ff872d7,bf1146d7-3da3-40ca-9c90-9396536acd70,cat,Heda,Siamese,Reptiles
2,15151d97-38e4-4ab4-b058-a2285573ba46,68b03382-578c-40b5-ac85-d6de6bc9fbee,dog,Vergil,Labrador Retriever,Reptiles
3,60870eef-36a4-44c2-a041-bfa81521e080,a4b7c257-3e73-4840-8443-64366ef87002,dog,Charissa,Parakeet,Birds
4,81d3f207-c224-4825-97d4-7c0f98daadad,3827c909-9a02-478c-81c5-d64817a3e8ae,bird,Levy,Siamese,Cats


In [10]:
customer_window = Window.partitionBy("email").orderBy("id")
customer_map = dim_customer \
    .withColumn("row_num", row_number().over(customer_window)) \
    .filter(col("row_num") == 1) \
    .select("id", "email")

print(customer_map.toPandas().shape)

seller_window = Window.partitionBy("email").orderBy("id")
seller_map = dim_seller \
    .withColumn("row_num", row_number().over(seller_window)) \
    .filter(col("row_num") == 1) \
    .select("id", "email")

print(seller_map.toPandas().shape)

store_window = Window.partitionBy("name", "email").orderBy("id")
store_map = dim_store \
    .withColumn("row_num", row_number().over(store_window)) \
    .filter(col("row_num") == 1) \
    .select("id", "name", "email")

print(store_map.toPandas().shape)

product_window = Window.partitionBy("name", "price", "category", "release_date", "expiry_date").orderBy("id")
product_map = dim_product \
    .withColumn("row_num", row_number().over(product_window)) \
    .filter(col("row_num") == 1) \
    .select("id", "name", "price", "category", "release_date", "expiry_date")

print(product_map.toPandas().shape)

fact_sales = mock_data.alias("m") \
    .join(customer_map.alias("cm"), col("m.customer_email") == col("cm.email"), "inner") \
    .join(seller_map.alias("sm"), col("m.seller_email") == col("sm.email"), "inner") \
    .join(product_map.alias("pm"),
          (col("m.product_name") == col("pm.name")) & 
          (col("m.product_price") == col("pm.price")) & 
          (col("m.product_category") == col("pm.category")) &
          (col("m.product_release_date") == col("pm.release_date")) &
          (col("m.product_expiry_date") == col("pm.expiry_date")),
          "inner") \
    .join(store_map.alias("stm"),
          (col("m.store_name") == col("stm.name")) & 
          (col("m.store_email") == col("stm.email")),
          "inner") \
    .select(
        col("m.sale_date").alias("sale_date"),
        col("cm.id").alias("sale_customer_id"),
        col("sm.id").alias("sale_seller_id"),
        col("pm.id").alias("product_id"),
        col("stm.id").alias("store_id"),
        col("m.sale_quantity").alias("sale_quantity"),
        col("m.sale_total_price").alias("sale_total_price")
    )


fact_sales = fact_sales.withColumn("id", expr("uuid()"))
fact_sales = fact_sales.selectExpr(
    "id",
    "sale_date",
    "sale_customer_id",
    "sale_seller_id",
    "product_id",
    "store_id",
    "sale_quantity",
    "sale_total_price"
)


print(fact_sales.toPandas().shape)
fact_sales.toPandas().head()

(10000, 2)
(10000, 2)
(10000, 3)
(10000, 6)
(10000, 8)


,id,sale_date,sale_customer_id,sale_seller_id,product_id,store_id,sale_quantity,sale_total_price
0,103ba651-a3ac-4bd1-b9d5-fffe3ecf7294,2021-11-28,d8959b4b-b682-4435-84c3-444b85a5ea76,15d2f20f-d4a4-4039-89a9-9825fd6e4ca1,01e89e1b-48f3-4d1f-9492-67c5450d0456,8fcd4a2d-7dc0-475d-8502-a016d8405868,3,42.79
1,d66acf8d-ca23-424d-8166-5da06d5ee8b5,2021-08-08,17492ca0-dc77-4e59-84c6-a9872b012968,744d6fe6-43b6-4999-bcf1-5217ec0885ef,602d972c-1755-4899-af7f-82b7f66c6ae0,9d70d924-469c-4b56-813b-e64a6bae14c8,9,272.28
2,a24b5e59-e41f-4e94-b883-0bdafba6a154,2021-10-27,3faf6ba7-c930-4606-a637-204158877b57,5ab86ce5-b01a-448e-a8e7-4fa2cbc325d3,6ed8de07-06ff-438f-a8aa-d451d353f26b,e9c152f6-d67a-47d9-bacc-7c26b6b566b6,7,341.96
3,bc06041a-41ec-402e-bb9c-4b48e4ef0d49,2021-09-24,8024839f-a18e-4e4b-a7c8-8cc8d4433b1b,8a95ba4c-14da-4ea3-8748-63202fd38a63,7a0aa08a-35cf-4f80-9a1a-993cc6563600,0cae4fa3-e591-43e6-93dd-c4cf9befcd64,6,408.89
4,e11b2717-ba38-43da-83f6-b72925e82b3f,2021-06-08,81cba2b8-2525-43d0-8f03-3ce4d860d913,40122c6f-e69f-49ef-8d17-8ab793972c9f,a2eaa65d-e714-47a3-a8df-c4ecf2ccdb5c,57a90a9f-2c49-4263-a515-75e6f35115ac,8,15.46


In [11]:
def write_to_postgres(df, table_name):
    df.write.jdbc(
        url=pg_jdbc_url,
        table=table_name,
        mode="append",
        properties=pg_properties,
    )

write_to_postgres(dim_customer, "dim_customer")
write_to_postgres(dim_seller, "dim_seller")
write_to_postgres(dim_supplier, "dim_supplier")
write_to_postgres(dim_store, "dim_store")
write_to_postgres(dim_product, "dim_product")
write_to_postgres(dim_pet, "dim_pet")

In [12]:
write_to_postgres(fact_sales, "fact_sales")

### Отчеты в clickhouse

#### Витрина продаж по продуктам

In [13]:
product_sales = fact_sales.alias("fs") \
    .join(dim_product.alias("p"), col("fs.product_id") == col("p.id")) \
    .groupBy("product_id", "name", "category") \
    .agg(
        sum("sale_quantity").alias("total_quantity"),
        sum("sale_total_price").alias("total_revenue"),
        avg("rating").alias("avg_rating"),
        avg("reviews").alias("avg_reviews")
    ) \
    .withColumnRenamed("name", "product_name") \
    .withColumnRenamed("category", "product_category")

product_sales.write.jdbc(ch_jdbc_url, "report_sales_by_product", mode="append", properties=ch_properties)

product_sales.toPandas().head()

,product_id,product_name,product_category,total_quantity,total_revenue,avg_rating,avg_reviews
0,9ddacf46-9c4b-4a8d-89d5-803cd6bb70d2,Bird Cage,Cage,9,204.32,1.30000,405.0
1,ef4bbc84-252e-4fb7-b9d8-b5552fcc7d53,Bird Cage,Toy,4,127.34,2.00000,231.0
2,3c4fd917-9f94-4875-b602-e27e5cd4e9ab,Bird Cage,Cage,1,133.62,2.10000,225.0
3,a1bcc603-391d-4621-bcab-37c4d7096744,Bird Cage,Food,2,447.72,4.20000,531.0
4,46a5ffff-78f5-4c11-badd-5185ecfb065b,Bird Cage,Food,9,90.05,1.40000,43.0


#### Витрина продаж по клиентам

In [14]:
customer_sales = fact_sales.alias("fs") \
    .join(dim_customer.alias("c"), col("fs.sale_customer_id") == col("c.id")) \
    .groupBy("sale_customer_id", "first_name", "last_name", "country") \
    .agg(
        sum("sale_total_price").alias("total_spent"),
        (sum("sale_total_price") / sum("sale_quantity")).alias("avg_order_value")
    ) \
    .withColumnRenamed("sale_customer_id", "customer_id")

customer_sales.write.jdbc(ch_jdbc_url, "report_sales_by_customer", mode="append", properties=ch_properties)

customer_sales.toPandas().head()

,customer_id,first_name,last_name,country,total_spent,avg_order_value
0,18da214e-42b1-41ad-80be-6df2285b1c9b,Ephrayim,Moralis,China,213.35,26.66875000000000000000
1,bc78ad14-fa17-4372-bbaa-f656150c71eb,Tiff,Frier,Indonesia,186.53,23.31625000000000000000
2,bf1558f5-bc9d-4ae2-a5fb-940d8f934bd2,Merrick,Nussgen,Ghana,102.10,11.34444444444444444444
3,b75fcaa8-1aae-435f-9d9b-ab32284446ad,Lanita,Belbin,China,389.52,43.28000000000000000000
4,0bb40572-818e-468d-ad03-42c5d5182504,Alix,Bartosch,Philippines,224.43,224.43000000000000000000


#### Витрина продаж по времени

In [15]:
time_sales = fact_sales \
    .withColumn("year", year("sale_date")) \
    .withColumn("month", month("sale_date")) \
    .groupBy("year", "month") \
    .agg(
        sum("sale_total_price").alias("monthly_revenue"),
        (sum("sale_total_price") / sum("sale_quantity")).alias("avg_order_value")
    )

time_sales.write.jdbc(ch_jdbc_url, "report_sales_by_time", mode="append", properties=ch_properties)

time_sales.toPandas().head()

,year,month,monthly_revenue,avg_order_value
0,2021,8,221275.78,45.92689497716894977169
1,2021,6,215042.80,48.45488958990536277603
2,2021,5,211764.86,47.57691754661873736239
3,2021,10,228743.32,45.96931672025723472669
4,2021,11,200154.69,46.58010006981615080289


#### Витрина продаж по магазинам

In [16]:
store_sales = fact_sales.alias("fs") \
    .join(dim_store.alias("s"), col("fs.store_id") == col("s.id")) \
    .groupBy("store_id", "name", "city", "country") \
    .agg(
        sum("sale_total_price").alias("total_revenue"),
        (sum("sale_total_price") / sum("sale_quantity")).alias("avg_order_value")
    ) \
    .withColumnRenamed("name", "store_name")

store_sales.write.jdbc(ch_jdbc_url, "report_sales_by_store", mode="append", properties=ch_properties)

store_sales.toPandas()

,store_id,store_name,city,country,total_revenue,avg_order_value
0,9443524e-9caa-45e8-847c-e4f9ce8d3f59,Divape,Póvoa do Valado,China,233.35,58.33750000000000000000
1,3140868d-2526-469f-b833-d8590e691254,Agivu,Sulang Tengah,Philippines,249.80,49.96000000000000000000
2,428646d7-832d-43a3-b9cf-6a5955db1920,Tagpad,Nantes,France,405.56,405.56000000000000000000
3,420f2a1a-5956-4e33-b04f-d84cc0c0386c,Skibox,Krahës,China,114.92,11.49200000000000000000
4,aaaddda6-ef5b-4326-ac9a-0c1ae91b0f0d,Mynte,Bom Despacho,Argentina,57.14,5.71400000000000000000
...,...,...,...,...,...,...
9995,682d26c6-877a-470f-ba81-5710a4747279,Shuffletag,Babao,Kyrgyzstan,132.83,18.97571428571428571429
9996,79a2ef37-5185-42f6-acd9-d51f1b7b69e8,Gigazoom,Karsin,China,375.56,62.59333333333333333333
9997,61e662cf-10fa-46c3-b289-47ac50f7a108,Blogspan,Meliau,China,218.00,72.66666666666666666667
9998,5ffc60f3-159e-4d8c-b071-5134d29766e4,Photolist,Kuantan,Russia,366.85,366.85000000000000000000


#### Витрина продаж по поставщикам

In [17]:
supplier_sales = fact_sales.alias("fs") \
    .join(dim_product.alias("p"), col("fs.product_id") == col("p.id")) \
    .join(dim_supplier.alias("s"), col("p.supplier_id") == col("s.id")) \
    .groupBy("s.id", "s.name", "s.country") \
    .agg(
        sum("fs.sale_total_price").alias("total_revenue"),
        avg("p.price").alias("avg_price")
    ) \
    .withColumnRenamed("id", "supplier_id") \
    .withColumnRenamed("name", "supplier_name")

supplier_sales.write.jdbc(ch_jdbc_url, "report_sales_by_supplier", mode="append", properties=ch_properties)

supplier_sales.toPandas().head()

,supplier_id,supplier_name,country,total_revenue,avg_price
0,8e6b7551-94d0-4a18-931d-d6c2c2e3be64,Riffpath,Germany,148.24,13.330000
1,4ffa5ad3-a126-4d9d-9052-3260ae8f0f91,Dabvine,Indonesia,182.08,25.870000
2,d925dd08-5f73-4aa4-a48f-8a86134b45b4,Mymm,Sweden,278.78,27.310000
3,5d949542-ed48-477f-98ef-7e8e435462ab,Gigashots,China,45.74,32.340000
4,37efe8d7-a7f9-4e80-aa27-c8555e7b8b8b,Flashspan,Greece,252.59,34.160000


#### Витрина качества продукции

In [18]:
product_quality = fact_sales.alias("fs") \
    .join(dim_product.alias("p"), col("fs.product_id") == col("p.id")) \
    .groupBy("product_id", "name") \
    .agg(
        avg("rating").alias("avg_rating"),
        sum("sale_quantity").alias("total_quantity"),
        sum("reviews").alias("total_reviews")
    ) \
    .withColumnRenamed("name", "product_name")

product_quality.write.jdbc(ch_jdbc_url, "report_product_quality", mode="append", properties=ch_properties)

product_quality.toPandas().head()

,product_id,product_name,avg_rating,total_quantity,total_reviews
0,e49b1a22-691e-4aed-a22a-fd042d5d0807,Bird Cage,1.00000,6,293
1,4fbe02e7-91de-4a0b-bdac-5285cc6048d5,Bird Cage,2.30000,10,490
2,03b590bd-c24d-4757-b507-933977b0c841,Bird Cage,1.50000,8,778
3,d59f853d-a8e0-46a0-9bc9-75026ad74152,Bird Cage,3.10000,2,623
4,f31175c7-1711-4924-b89c-480a8010350c,Bird Cage,1.10000,5,245
